# Video Output Control Panel

This notebook uses the Xilinx Video Mixer IP to display up to four windows on the HDMI output. It assumes that a camera is connected via HDMI input generating 1080p video and outputs 4K

First perform a series of imports and download the overlay. The `stat_buffer` variable should be set to the output of the second cell of the `Stats` notebook. This allows for the graph showing CPU usage and power to be displayed on the HDMI output

In [1]:
import functools
import threading
from pynq import Xlnk
from demo_drivers import *
from pynq.overlays.base import BaseOverlay
import asyncio
from pynq.lib.video import *

xlnk = Xlnk()
ol = BaseOverlay('./base_logo.bit')
stat_buffer = 0x62a00000

Next set up the HDMI input pipeline to `UYVY` format and start a loop to service any `asyncio` coroutines we need to launch.

In [2]:

hdmi_in = ol.video.hdmi_in
dma = ol.video.axi_vdma

hdmi_in.frontend.start()
hdmi_in.colorspace = COLOR_IN_YCBCR
hdmi_in.pixel_pack.write(0x10, 5)

loop = asyncio.get_event_loop()
def run_loop():
    loop.run_forever()
    
asyncio_thread = threading.Thread(target=run_loop)
asyncio_thread.start()

Next up are some helper classes for interacting with the various IP in the design, namely the HDMI input, 2D filter and optical flow unit. These classes simplify the implementation of the final control panel by providing a unified API.

In [3]:
import cv2

class HDMIWrapper:
    def __init__(self, dma):
        self._listeners = set()
        self._channel = dma.readchannel
        self._channel.mode = VideoMode(1920,1080,16)
        self.running = True
        
    def start(self):
        self.running = True
        self.task = asyncio.run_coroutine_threadsafe(self._run_thread(), asyncio.get_event_loop())
    
    def stop(self):
        self.running = False
    
    def add_listener(self, listener):
        self._listeners.add(listener)
    
    def remove_listener(self, listener):
        if listener in self._listeners:
            self._listeners.remove(listener)

    async def _run_thread(self):
        channel = self._channel
        channel.start()
        prev_frame = None
        while self.running:
            frame = await channel.readframe_async()
            if prev_frame is not None:
                for l in self._listeners:
                    l.next_frame(frame, prev_frame)
            prev_frame = frame
        
class VDMAListener:
    def __init__(self, dma, layer):
        self._channel = dma.writechannel
        dma.writechannel.mode = VideoMode(1920,1080,16)
        dma.writechannel.start()
        self._layer = layer
        layer.width = 1920
        layer.height = 1080
        
    def next_frame(self, frame, prev_frame):
        self._channel.setframe(frame)
        
    def show(self):
        self._layer.enable()
        
    def hide(self):
        self._layer.disable()
        
class FlowDriver:
    def __init__(self, ip, layer):
        self._ip = ip
        self._layer = layer
        self.out_frame = xlnk.cma_array(shape=(1080,1920,4), dtype='u1')
        self._layer.buffer = self.out_frame.physical_address
        self._layer.width = 1920
        self._layer.stride = 1920 * 4
        self._layer.height = 1080
        
    def show(self):
        self._layer.enable()
    
    def hide(self):
        self._layer.disable()
        
    def next_frame(self, frame, prev_frame):
        self._ip.process(frame, prev_frame, self.out_frame)
    
class FilterDriver:
    def __init__(self, ip, layer):
        self._ip = ip
        self._layer = layer
        self.out_frame = xlnk.cma_array(shape=(1080,1920,4), dtype='u1')
        self._layer.buffer = self.out_frame.physical_address
        self._layer.width = 1920
        self._layer.stride = 1920 * 4
        self._layer.height = 1080
    
    def show(self):
        self._layer.enable()
    
    def hide(self):
        self._layer.disable()
    
    def next_frame(self, frame, prev_frame):
        self._ip.process(frame, self.out_frame)

Next we need to set up all of the devices and start the HDMI output

In [4]:
mixer = ol.video.hdmi_out.v_mix_0
tpg = ol.video.hdmi_out.v_tpg_0

hdmi = HDMIWrapper(dma)
hdmi_display = VDMAListener(dma, mixer.layers[1])
hdmi.add_listener(hdmi_display)
filter2d = FilterDriver(ol.filter_pipeline_0, mixer.layers[2])
flow = FlowDriver(ol.optical_flow_0, mixer.layers[3])

stat_layer = mixer.layers[4]
stat_layer.buffer = stat_buffer
stat_layer.width = 960
stat_layer.height = 540
stat_layer.scale = 1
stat_layer.stride = 960 * 4

logo_layer = mixer.layers[5]
logo_layer.width = 256
logo_layer.height = 81
logo_layer.scale = 1

hdmi_out = ol.video.hdmi_out.frontend
hdmi_out.mode = VideoMode(3840, 2160, 24, 30)
hdmi_out.start()

mixer.width = 3840
mixer.height = 2160
tpg.width = 3840
tpg.height = 2160
tpg.pattern = 0xD
mixer.enable_layer(0)
mixer.start()
tpg.start()

Frequency: 297000000


Finally we can build up the control panel. Each layer has adjustable X, Y and transparency and can be enabled or disabled.

In [5]:
import ipywidgets as widgets

def handle_x(change):
    layer = change['owner'].layer
    layer.x = change["owner"].value

def handle_y(change):
    layer = change['owner'].layer
    layer.y = change['owner'].value
    
def handle_alpha(change):
    layer = change['owner'].layer
    layer.alpha = int(change['owner'].value * 256)
    
class LayerControls:
    def __init__(self, name, layer, max_x=1920, max_y=1080):
        self._xslider = widgets.IntSlider(value=0, min=0, max=max_x, description="X Position")
        self._xslider.layer = layer
        self._xslider.observe(handle_x)
        self._yslider = widgets.IntSlider(value=0, min=0, max=max_y, description="Y Position")
        self._yslider.layer = layer
        self._yslider.observe(handle_y)
        self._alpha = widgets.FloatSlider(value=1, min=0, max=1, step=0.01, description="Opacity")
        self._alpha.layer = layer
        self._alpha.observe(handle_alpha)
        self._name = widgets.HTML(f"<h3>{name}</h3>", layout={"width": "150px"})
        self._vbox = widgets.VBox([self._xslider, self._yslider, self._alpha])
        self._options.observe(self._status_change)
        self.hbox = widgets.HBox(
            [self._name, self._options, self._vbox], layout={"border": "solid 2px", "padding": "10px"})
        
class SourceControls(LayerControls):
    def __init__(self, name, layer, source, **kwargs):
        self._options = widgets.RadioButtons(
            options=["Off", "Hidden", "Visible"], description="Status", layout={"width": "250px"})
        self._layer = layer
        self._source = source
        super().__init__(name, layer, **kwargs)

    def _status_change(self, new_option):
        setting = self._options.value
        if setting == "Off":
            self._source.stop()
            self._layer.disable()
        elif setting == "Hidden":
            if not self._source.running:
                self._source.start()
            self._layer.disable()
        else:
            if not self._source.running:
                self._source.start()
            self._layer.enable()
    
class ProcessingControls(LayerControls):
    def __init__(self, name, layer, processing, **kwargs):
        self._options = widgets.RadioButtons(options=["Off", "On"], description="Status", layout={"width": "250px"})
        self._layer = layer
        self._processing = processing
        super().__init__(name, layer, **kwargs)
        
    def _status_change(self, new_option):
        if self._options.value == "On":
            self._layer.enable()
            if self._processing:
                hdmi.add_listener(self._processing)
        else:
            self._layer.disable()
            if self._processing:
                hdmi.remove_listener(self._processing)
    
control_panel = widgets.VBox([SourceControls("Webcam", mixer.layers[1], hdmi).hbox, 
                              ProcessingControls("Filter", mixer.layers[2], filter2d).hbox,
                              ProcessingControls("Optical Flow", mixer.layers[3], flow).hbox,
                              ProcessingControls("Status", mixer.layers[4], None).hbox,
                              ProcessingControls("Logo", mixer.layers[5], None, max_x=3328, max_y=1998).hbox])

Show the control panel

In [6]:
control_panel

![Video Pipeline](VideoPipeline.png)

In [7]:
ol?
